In [7]:
import os
from typing import List, Any

import numpy as np
from IPython.lib.display import Audio
from numpy import ndarray, dtype
from scipy.signal import sawtooth
from ipywidgets import widgets, interact

### CONSTANTS ###
NOTES = {b: a for a, b in enumerate(('C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'))}
NOTES.update({b: a for a, b in enumerate(('B#', 'Db', 'D', 'Eb', 'Fb', 'E#', 'Gb', 'G', 'Ab', 'A', 'Bb', 'Cb'))})
REST = "X"

In [21]:
class Synth:
    result: ndarray[Any, dtype[Any]]
    sequence: list[tuple[tuple[float, ...], float]]
    global_rate: int

    def __init__(self, global_rate=48000):
        self.file = None
        self.global_rate = global_rate
        self.sequence = []
        self.length = 0
        self.result = np.array([])
        self.bpm = 120
        self.rev = False
        self.adsr = [0, 0, 1, 0]
        
        
    def set_sequence(self, file_path):
        """
        Parses a file containing notes and durations and sets the sequence for playback.
        
        :param file_path: Path to a txt file containing the sequence information.
        """
        if not file_path or "option" in file_path:
            return
        
        self.sequence = [] 
        self.file = file_path
        with open(file_path, 'r') as file:
            for line in file:
                parts = line.strip().split()
                self.sequence.append((tuple(self.note_to_freq(note) for note in parts[:-1]),  240 / self.bpm * float(parts[-1])))
                self.length += self.sequence[-1][1]
    
    def set_bpm(self, bpm):
        self.bpm = bpm
        # Have to reset timings
        self.set_sequence(self.file)
    
    def set_reversed(self, rev):
        self.rev = rev
    
    def set_adsr(self, *args):
        if len(args) == 4:
            self.adsr = args

    @staticmethod
    def note_to_freq(note: str) -> float:
        """
        Converts a string representation of a note to a floating point frequency.
        
        :param note: note in the form [CHARACTER(S)][OCTAVE] e.g. C#4
        :return: frequency associated with MIDI value
        """
        if note == REST:
            return 0
        letter = NOTES[note[:-2] if note.find('-') > -1 else note[:-1]]
        octave = int(note[-2:] if note.find('-') > -1 else note[-1:])
        pitch = 12 * (octave + 1) + letter
        return 440 * 2 ** ((pitch - 69) / 12)

    def calculate_env(self, dur, a=0, d=0, s=1, r=0):
        samples = int(self.global_rate * dur)
        attack = np.linspace(0, 1, int(self.global_rate * a))
        decay = np.linspace(1, s, int(self.global_rate * d))
        if dur < a + d:
            env = np.concatenate((attack, decay))
            release = np.linspace(env[-1], 0, int(self.global_rate*r))
            env = np.concatenate((env[:samples], release))
        else:
            sussy = np.full(int((dur - a - d) * self.global_rate), s)
            release = np.linspace(s, 0, int(self.global_rate*r))
            env = np.concatenate([attack, decay, sussy, release])
        return env

    def num_del(self, audio, ms, mix=0.5, delays=3):
        sec = ms / 1000.0
        samps = int(delays * sec * self.global_rate)
        pad = np.zeros(samps)
        dry = np.concatenate((audio, pad), axis=0)
        wet = np.zeros(dry.size)
        for delay in range(delays):
            wet += np.roll(dry, (delay+1)*int(sec*self.global_rate))/(1+delay)
        output = (1-mix)*dry + mix*wet
        return output

    def _bell_base_generator(self, ms=0, delays=0, attack=0, decay=5, sustain=0, release=2, mix=0.5, reverse=False):
        I = 2.234
        dsr = [decay, sustain, release]
        cur_samp = 0
        song = np.zeros(int(self.global_rate * (self.length + dsr[2])))

        for chord in self.sequence:
            dur = chord[1] + dsr[2]
            da_chord = np.zeros(int(self.global_rate * dur))
            t = np.linspace(0, dur, int(self.global_rate * dur))
            for freq in chord[0]:
                if freq == 0:
                    da_chord+=np.zeros(int(self.global_rate * (chord[1]+dsr[2])))
                    break
                mod = I * sawtooth(2 * np.pi * freq * (2) * (t + 1 / (freq * 8)), 0.5)
                fm = sawtooth(2 * np.pi * freq * (t + 1 / (freq * 4)) + mod, 0.5)
                da_chord += fm
            
            env = da_chord * self.calculate_env(chord[1], attack, *dsr)
            pad = np.concatenate((env, np.zeros(int(int(self.length * self.global_rate) - self.global_rate * chord[1]))))
            rolled = np.roll(pad, cur_samp)
            cur_samp += int(chord[1] * self.global_rate)
            song += rolled
        delayed = self.num_del(song, ms, mix=mix, delays=delays) 
        if reverse:
            delayed = delayed[::-1]
        return delayed
    
    def generate_audio(self):
        if self.file:
            return Audio(self._bell_base_generator(), rate=self.global_rate)
        else:
            print("File has not been selected.")
        

In [23]:
class SynthUI:
    text_files: list[str]
    synth: Synth

    def __init__(self, synth):
        self.synth = synth
        self.text_files = list(file for file in os.listdir('.') if file[-4:] == '.txt')
        
        self.bpm_slider = widgets.IntSlider(
            value=self.synth.bpm,
            min=60,
            max=240,
            step=1,
            description='BPM:',
            continuous_update=False
        )
        self.bpm_slider.observe(lambda delta: self.synth.set_bpm(delta['new']), 'value')
        
        # Reverse Toggle
        self.rev_toggle = widgets.ToggleButton(
            value=self.synth.rev,
            description='Reverse',
            tooltip='Reverse the sequence'
        )
        self.rev_toggle.observe(lambda delta: self.synth.set_reversed(delta['new']), 'value')
        
        # ADSR Sliders
        adsr_labels = [('Attack:', 5), ('Decay:', 5), ('Sustain:', 1), ('Release:', 5)]
        self.adsr_sliders = [
            widgets.FloatSlider(
                value=getattr(self.synth, 'adsr')[i][0],
                min=0,
                max=1,
                step=0.01,
                description=label,
                continuous_update=False
            ) for i, label in adsr_labels
        ]
        for slider in self.adsr_sliders:
            slider.observe(self.update_adsr)
        
        # File Dropdown
        self.file_dropdown = widgets.Dropdown(
            options=['Select an option'] + self.text_files,
            value='Select an option',
            description='Score file:',
        )
        # self.file_dropdown.observe(lambda delta: self.synth.set_sequence(delta['new']) if delta['new'] else None, 'value')
        interact(self.synth.set_sequence, file_path=self.file_dropdown)
        
        self.activate_button = widgets.ToggleButton(value=False, description="Generate audio")
        # self.activate_button.observe(lambda delta: self.activate())
        interact(self.activate, state=self.activate_button)
        
        # Display Widgets
        # display(self.bpm_slider, self.rev_toggle, *self.adsr_sliders, self.file_dropdown, self.activate_button)

    def update_adsr(self):
        self.synth.set_adsr(slider.value for slider in self.adsr_sliders)
    
    def activate(self, state):
        if state:
            res = self.synth.generate_audio()
            if res:
                display(res)
            self.activate_button.value = False

synth_ui = SynthUI(Synth())

TypeError: list indices must be integers or slices, not str

In [5]:
synth = Synth()
synth.set_sequence('arp.txt')
print(synth.sequence)
# Audio(synth.bell_base_generator(ms=100, decay = 0.25, sustain=0.25, release=1, delays=0, mix=0.5, reverse = True), rate=synth.global_rate)

G3           0.0625

Bb3          0.0625

D4           0.0625

Eb4          0.0625

G4           0.0625

Bb4    G3    0.0625

D5     G4    0.0625

Eb5    Bb3   0.0625

G5     D4    0.0625

Eb5    G4    0.0625

D5     Bb3   0.0625

Bb4    D4    0.0625

G4           0.0625

Eb4          0.0625

D4           0.0625

Bb3    Eb3   0.0625

G3     D3    0.125

[((195.99771799087463,), 0.125), ((233.08188075904496,), 0.125), ((293.6647679174076,), 0.125), ((311.1269837220809,), 0.125), ((391.99543598174927,), 0.125), ((466.1637615180899, 195.99771799087463), 0.125), ((587.3295358348151, 391.99543598174927), 0.125), ((622.2539674441618, 233.08188075904496), 0.125), ((783.9908719634985, 293.6647679174076), 0.125), ((622.2539674441618, 391.99543598174927), 0.125), ((587.3295358348151, 233.08188075904496), 0.125), ((466.1637615180899, 293.6647679174076), 0.125), ((391.99543598174927,), 0.125), ((311.1269837220809,), 0.125), ((293.6647679174076,), 0.125), ((233.08188075904496, 155.56349186104046), 